## 제 1고지 미분 자동 계산

### 1단계 산자로서의 변수

#### 1.1 변수란

- 데이터를 저장하는 상자와 같다

#### 1.2 Variable 클래스 구현

- DeZero에서 사용하는 변수라는 개념을 Variable이라는 이름의 클래스로 구현
- 클래스 이름의 첫글자는 보통 대문자
- 파이썬이 권장하는 코딩 규칙은 [PEP8](https://www.python.org/dev/peps/pep-0008)
    - PEP8 : 파이썬 개선 제안서, 파이썬 코드를 어떻게 구상할 지 알려주는 스타일 가이드

In [4]:
# steps/step01.py
class Variable:
    def __init__(self, data):
        self.data = data

- __init__에 주어진 인수를 인스턴스 변수 data에 대입
- Variable의 data에 보관

In [5]:
# steps/step01.py
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


- 이 예에서 상자에 넣는 데이터로 '넘파이 다차원 배열'을 사용
- x는 Variable 인스턴스 이며, 실제 데이터는 x안에 담겨 있음
- x는 데이터 자체가 아니라 데이터의 담는 상자 즉, 변수

- 머신러닝 시스템은 기본 데이터 구조로 '다차원 배열'을 사용
- DeZero의 Variable 클래스는 넘파이의 다차원 배열만 취급
- 넘파이 배열은 np.array 함수로 생성 가능
- numpy.ndarray 인스턴스를 ndarray 인스턴스로 부름

In [7]:
# steps/step01.py
x.data = np.array(2.0) # x에 새로운 데이터 대입
print(x.data)

2.0


#### 1.3 넘파이 다차원 배열

- 다차원 배열은 숫자 등의 원소가 일정하게 모여 있는 데이터 구조
- 다차원 배열에서 원소의 순서에는 방향이 있고, 이 방향을 차원(dimension) 혹은 축(axis)이라고 함
- 0차원, 1차원, 2차원 배일이 있는데, 차례대로 스칼라(scalar), 벡터(vector), 행렬(matrix)이라고 함
    - 스칼라는 단순히 하나의 수
    - 벡터는 하나의 축을 따라 숫자가 늘어서 있음
    - 행렬은 축이 두 개
- 다차원 배열을 0차원 텐서(tensor), 1차원 텐서, 2차원 텐서라고도 함 

In [9]:
# ndim 은 'number of dimensions'의 약자로, 다차원 배열의 '차원 수'를 뜻함
import numpy as np
x = np.array(1)
x.ndim

0

In [10]:
x = np.array([1, 2, 3])
x.ndim

1

In [16]:
x = np.array([[1, 2, 3],
              [4, 5, 6]])
x.ndim

2

### 2단계 변수를 낳는 함수

#### 2.1 함수란

- 어떤 변수로부터 다른 변수로의 대응 관계를 정한 것

#### 2.2 Function 클래스 구현

- Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스턴스를 출력
- Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 있음

In [20]:
class Function:
    def __call__(self, input):
        x = input.data # 데이터를 꺼냄
        y = x ** 2 # 실제 계산
        output = Variable(y) # Variable 형태로 되돌림
        return output

- `__call__` 메서드는 파이썬의 특수 메서드
- f = Function() 형태로 함수의 인스턴스를 변수 f에 대입해 둠
- f(...)형태로 `__call__` 메서드를 호출할 수 있음

#### 2.3 Function 클래스 이용

In [22]:
x = Variable(np.array(10))
f = Function()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


- Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현
- 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현

In [24]:
# steps/step02.py
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)
        return output
    def forward(self, x):
        raise NotImplementedError()

- NotImplementedError()는 '이 메서드는 상속하여 구현해야 한다'는 사실을 알려주는 예외처리

In [25]:
# steps/step02.py
# 입력값을 제곱하는 클래스 구현
class Square(Function):
    def forward(self, x):
        return x ** 2

In [26]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


### 3단계 함수 연결

#### 3.1 Exp 함수 구현

- 오일러의 수, 네이피어 상수 구현

In [28]:
# steps/step03.py
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

#### 3.2 함수 연결

In [29]:
# steps/step03.py
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


- 여러 함수로 구성된 함수를 '합성 함수'라고 함

### 4단계 수치 미분

#### 4.1 미분이란

- 미분은 변화율
- 극한으로 짧은 시간(순간)에서의 변화량
- 도함수 : 함수 f(x)가 주어졌을 때 함수의 정의역에 속하는 각각의 x의 값에 미분계수가 하나씩 대응되는 함수

###

#### 4.2 수치 미분 구현

- 컴퓨터는 극한을 취급할 수 없음
- h = 0.0001(=1e-4)과 같은 매우 작은 값으로 대체
- 미세한 차이를 이용하여 함수의 변화량을 구하는 방법을 '수치 미분'이라 함
- 수치 미분은 작은 값을 사용하여 '진정한 미분'을 근사
- 어쩔수 없이 오차가 포함
- 근사 오차를 줄이는 방법으로 '중앙차분'을 씀
    - 중앙차분은 f(x)와 f(x+h)의 차이를 구한는 대신에 f(x-h)와 f(x+h)의 차이를 구함
- 전진차분보다 중앙차분이 진정한 미분값에 가깝다는 사실은 테일러 급수를 이용해 증명가능

![그림 4-2 진정한 미분, 전진차분, 중앙차분 비교](./image/4-2.png)

- 중앙차분을 이용하여 수치 비분을 계산하는 함수 numerical_diff(f, x, eps=1e-4)을 구현
- f는 Function의 인스턴스, x는 미분을 계산하는 변수로 Variable 인스턴스, eps은 작은 값

In [35]:
# steps/step04.py
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [36]:
# steps/step04.py
# Square 클래스를 대상으로 미분
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


#### 4.3 합성 함수의 미분

- 합성 함수를 미분해보자

In [38]:
# steps/step04.py
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


#### 4.4 수치 미분의 문제점

- 수치 미분의 결과에는 오차가 포함
- 대부분의 경우 오차는 매우 작지만 어떤 계산이냐에 따라 커질 수 있음
- 수치 미분의 결과에 오차가 포함되기 쉬운 이유는 주로 '자릿수 누락' 때문
- 계산량이 많다는 점도 심각한 문제
- 그래서 등장한 것이 '역전파'
- 역전파는 복잡한 알고리즘이라서 구현하면서 버그가 섞여 들어가기 쉬움
- 역전파를 정확하게 구현했는지 확인하기 위해 수치 미분의 결과를 이용하는 방식을 '기울기 확인'이라함
- 기울기 확인 : 단순히 수치 미분 결과와 역전파의 결과를 비교

### 5단계 역전파 이론

- 역전파을 이용하면 미분을 효율적으로 계산할 수 있고 결과값의 오차도 적음

#### 5.1 연쇄 법칙

- 역전파를 이해하는 열쇠는 '연쇄 법칙(chain rule)'
- 연쇄 법칙에 따르면 합성 함수의 미분은 구성 함수 각각을 미분한 후 곱한 것과 같음

#### 5.2 역전파 원리 도출

- 머신러닝은 주로 대량의 매개변수를 입력받아서 마지막에 '손실 함수(loss function)'를 거쳐 출력을 내는 형태
- 손실 함수의 출력은 단일한 스칼라값이며, 이 값이 '중요 요소'
- 머신러닝은 주로 대량의 매개변수를 입력받아서 마지막에 '손실 함수(loss function)'를 거쳐 출력을 내는 형태
- 미분값을 출력에서 입력 방향으로 전파하면 한 번의 전파만으로 모든 매개변수에 대한 미분을 계산할 수 있음

#### 5.3 계산 그래프로 살펴보기

- 변수는 '통상값'과 '미분값'이 존재
- 함수는 '통상 계산(순전파)'과 '미분값을 구하기 위한 계산(역전파)'이 존재
- 역전파 시에는 순전파시에 이용한 데이터가 필요, 따라서 역전파를 구현하려면 먼저 순전파를 하고, 이때 각 함수가 입력 변수의 값을 기억해둬야함

![그림 5-5](./image/5-5.png)

### 6단계 수동 역전파

- 역전파의 구동 원리를 설명
- Variable과 Function 클래스를 확장하여 역전파를 이용한 미분 구현

#### 6.1 Variable 클래스 추가 구현

- 통산값(data)과 더불어 그에 대응하는 미분값(grad)도 저장하도록 확장

In [46]:
# steps/step06.py
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

#### 6.2 Function 클래스 추가 구현

- 미분을 계산하는 역전파(backward 메서드)
- forward 메서드 호출 시 건네받은 Variable 인스턴스 유지

In [48]:
# steps/step06.py
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억(보관)함
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

- `__call__` 메서드에서 입력된 input을 인스턴스 변수인 self.input에 저장
- backward 메서드에서 함수에 입력한 변수가 필요할 때 self.input에서 가져와 사용 

#### 6.3 Square 와 Exp 클래스 추가 구현

In [79]:
# steps/step06.py
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [80]:
# steps/step06.py
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

- 순전파 코드

In [81]:
# steps/step06.py
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

- 역전파 코드

In [82]:
# steps/step06.py
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


### 7단계 역전파 자동화

- 순전파를 한 번만 해주면 어떤 계산이라도 상관없이 역전파가 자동으로 이루어지는 구조 만들기
- Define-by-Run이란 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식으로, '동적 계산 그래프'라고 함

#### 7.1 역전파 자동화의 시작

- 역전파 자동화로 가는 길은 변수와 함수의 '관계'를 이해하는 데서 출발
- 함수 관점에서 변수는 '입력'과 '출력'에 쓰임
- 변수 과점에서 함수는 '창조자' 혹은 '부모'

- 일반적인 순전파가 이루어지는 시점에 '관계'를 맺어줌

In [83]:
# steps/step07.py
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # creator 설정
        self.creator = func

- creator라는 인스턴스 변수 추가, creator 설정을 위한 set_creator 메서드 추가

In [84]:
# steps/step07.py
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # Set parent(function)
        self.input = input
        self.output = output  # Set output
        return output

- 순전파를 계산하면 그 결과로 output이라는 Variable 인스턴스가 생성
- oupput이 creator를 기억

In [85]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

- assert문은 조건을 충족하는지 여부를 확인하는 데 사용

#### 7.2 역전파 도전!

- 1. 함수를 가져온다.
- 2. 함수의 입력을 가져온다.
- 3. 함수의 backward 메서드를 호출한다.

In [87]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다.
b = C.input # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다.

In [88]:
B = b.creator # 1. 함수를 가져온다.
a = B.input # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호출한다.

In [89]:
A = a.creator # 1. 함수를 가져온다.
x = A.input # 2. 함수의 입력을 가져온다.
x.grad = A.backward(a.grad) # 3. 함수의 backward 메서드를 호출한다.
print(x.grad)

3.297442541400256


#### 7.3 backward 메서드 추가

- 위 반복작업을 자동화할 수 있도록 Variable 클래스에 backward 메서드 추가

In [91]:
# steps/step07.py
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. Get a function
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward
            x.backward()

- backward 메서드가 재귀적으로 호출면서 자동화

In [93]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### 8단계 재귀에서 반복문으로

- 처리 효율을 개선하고 확장을 대비해 backward 메서드의 구현 방식을 변경

#### 8.1 현재의 Variable 클래스